In [ ]:
!unzip HappyImages.zip

In [ ]:
!unzip SurprisedImages.zip

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import random
from sklearn.model_selection import train_test_split

In [ ]:
surprisedImages = []
for i in range(150):
  im = cv2.imread(f'SurprisedImages/IMAGE_{i}.png', cv2.IMREAD_GRAYSCALE)
  im = cv2.resize(im, dsize=(100, 100))
  if not (im[0][0] == 0 and im[0][1] == 0):
    surprisedImages.append((im, 1))

In [ ]:
happyImages = []
for i in range(150):
  im = cv2.imread(f'HappyImages/IMAGE_{i}.png', cv2.IMREAD_GRAYSCALE)
  im = cv2.resize(im, dsize=(100, 100))
  if not (im[0][0] == 0 and im[0][1] == 0):
    happyImages.append((im, 0))

In [ ]:
totalImages = happyImages + surprisedImages

In [ ]:
random.shuffle(totalImages)

In [ ]:
len(totalImages)

447

In [ ]:
X = [x[0] for x in totalImages]
X = np.array(X)
X = np.reshape(X, (len(totalImages), 100, 100, 1))
Y = [x[1] for x in totalImages]
Y = np.array(Y)
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2)

In [ ]:
x1 = tf.keras.layers.Input((100, 100, 1))
x2 = tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu')(x1)
x3 = tf.keras.layers.MaxPooling2D((2, 2))(x2)
x4 = tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu')(x3)
x5 = tf.keras.layers.MaxPooling2D((2, 2))(x4)
x6 = tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu')(x5)
x7 = tf.keras.layers.MaxPooling2D((2, 2))(x6)
f = tf.keras.layers.Flatten()(x7)
d1 = tf.keras.layers.Dense(32, activation = 'relu')(f)
d2 = tf.keras.layers.Dense(16, activation = 'relu')(d1)
d3 = tf.keras.layers.Dense(1, activation = 'sigmoid')(d2)
model = tf.keras.models.Model(inputs=x1, outputs=d3)

In [ ]:
model.summary()
model.compile(loss='binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 16)        4624  

In [ ]:
model.fit(xtrain, ytrain, batch_size = 10, epochs = 100)

In [ ]:
totalCorrect = 0
predictions = model.predict(xtest)
for i in range(len(predictions)):
  binary = np.round_(predictions[i])
  if binary == ytest[i]:
    totalCorrect += 1
print(totalCorrect / len(ytest))

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
model_path = "/content/drive/My Drive/EmotionModel.h5"
model.save(model_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
